In [1]:
import re

def extract_video_id(url):
    pattern = r"(?:v=|\/)([0-9A-Za-z_-]{11})"
    match = re.search(pattern, url)
    return match.group(1) if match else None


In [2]:
video_id = extract_video_id("https://youtu.be/noecHp7uZrY?si=GB3pFXmzbPGnbbE5")

print("ID is : " , video_id)

ID is :  noecHp7uZrY


In [3]:
!.venv\Scripts\activate


In [4]:



from youtube_transcript_api import YouTubeTranscriptApi


def get_transcript(video_id: str):
    


        api = YouTubeTranscriptApi()


        transcript_data = api.fetch(video_id=video_id)


        full_text = " ".join(item.text for item in transcript_data)
        
        
        
        return full_text

        


In [5]:
transcript = get_transcript(video_id)

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


c:\Users\Nirav Rupapara\Desktop\YouTube-ChatBot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([transcript])

In [8]:
text_chunks = [chunk.page_content for chunk in chunks]

In [9]:
len(text_chunks)

3

In [10]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")  # Free embedding model

  # embedding vector size: 384


In [12]:
from langchain_community.vectorstores import FAISS

In [13]:
embeddings = model.encode(text_chunks , convert_to_numpy=True)


In [14]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings



# Offline 22MB model
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# chunks = list of LangChain Document objects
vector_store = FAISS.from_documents(chunks, embeddings)


C:\Users\Nirav Rupapara\AppData\Local\Temp\ipykernel_23020\562646358.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [15]:
vector_store.index_to_docstore_id


{0: '118beb6e-85b6-4e48-b6a4-5eed752568af',
 1: 'fe15a8fa-1577-4010-8b95-f566fb7c86e4',
 2: '11334143-2e35-433a-a4bb-69fa2f336eeb'}

In [16]:
retriever = vector_store.as_retriever(
    search_type="similarity",  # or "mmr"
    search_kwargs={"k": 2}     # top 3 documents
)


In [17]:
query = "what is this video about?"
docs = retriever.invoke(query)

In [18]:
docs

[Document(id='11334143-2e35-433a-a4bb-69fa2f336eeb', metadata={}, page_content="me check wait let me check okay okay uh i have to make the open video false because if i not make it false it always open in my browser and we don't need it we just need the url of that video so if we make the open video false the video will not open in our browser and we get the url of that video and let's run again the program and this time it will work for example x if i enter the name get the url of that video and for checking that you can copy the url so okay follow link sorry for that uh you can open in your browser and it's that video we are going to play so that's it for today guys i hope you liked it i hope it helped you and if there is any uh query you can ask in comment section and if there is issue while running the program in your computer or in your laptop you can ask in the comment section i will show reply your queries and that's it for today guys bye bye take care"),
 Document(id='fe15a8fa-

In [19]:
from langchain_core.prompts import PromptTemplate



template = PromptTemplate(
    template="""
You are an AI assistant that answers questions strictly using the provided YouTube video transcript.

Instructions:
- Use ONLY the information from the transcript context below.
- If the answer is not present in the context, say:
  "The answer is not available in the provided video transcript."
- Keep the answer clear, concise, and easy to understand.
- Do NOT add external knowledge.

Transcript Context:
--------------------
{context}

Question:
---------
{question}

Answer:
"""
, input_variables=['context' , 'question']
)

In [23]:

from langchain_huggingface import HuggingFaceEndpoint , ChatHuggingFace

llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    temperature= 0,
    max_new_tokens= 512
    
)


model = ChatHuggingFace(llm=llm)

In [24]:
result = model.invoke("Hello")
print(result)

content=" Hello there! How can I help you today? Feel free to ask me any question or task you have in mind. I'm here to assist you. :-)" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 7, 'total_tokens': 41}, 'model_name': 'mistralai/Mistral-7B-Instruct-v0.2', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b1be0-182d-7341-a09a-160d9194562e-0' usage_metadata={'input_tokens': 7, 'output_tokens': 34, 'total_tokens': 41}


In [26]:
docs = retriever.invoke("What is this video about?")

print(len(docs))
print(docs[0].page_content[:300])


2
me check wait let me check okay okay uh i have to make the open video false because if i not make it false it always open in my browser and we don't need it we just need the url of that video so if we make the open video false the video will not open in our browser and we get the url of that video a


In [28]:
query = "what is this video about?"
docs = retriever.invoke(query)

In [30]:
prompt = template.invoke({'context':docs , 'question':query})

In [33]:
ans = model.invoke(prompt)

In [34]:
print(ans)

content=' This video appears to be about creating a Python script to obtain the URL of a YouTube video based on its name, without actually opening the video in the browser. The speaker in the video also mentions providing assistance in the comment section if there are any queries or issues.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 585, 'total_tokens': 639}, 'model_name': 'mistralai/Mistral-7B-Instruct-v0.2', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b1be3-b4f5-7b51-bb80-4996260b7b48-0' usage_metadata={'input_tokens': 585, 'output_tokens': 54, 'total_tokens': 639}
